In [1]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from geojson import dump
import json

In [2]:
# Read the sample CSV file
df = pd.read_csv("data/Total_Sales.csv")
df.head()

,PARCEL,Neighborhood,OWNER 1 FIRST NAME,OWNER 1 LAST NAME,OWNER 2 FIRST NAME,OWNER 2 LAST NAME,SITUS HOUSE NUMBER,SITUS STREET NAME,SITUS STREET ADDRESS,SITUS UNIT NUMBER,...,STORIES NO.,PROPERTY TAX,LMS-1ST MTG AMOUNT,LMS-1ST MTG TYPE,LMS-LENDER,LMS-TITLE COMPANY,ASSESSED TOTAL VALUE,ASSESSED LAND VALUE,ASSESSED IMPROVEMENT VALUE,SCHOOL DISTRICT 1
0,5559006020,West Hollywood,Thomas,Duke,NaN,NaN,8787,Shoreham,8787 Shoreham Dr #101,101,...,0.0,"$7,776.70",NaN,NaN,NaN,Equity Title Company,"$625,293.00","$480,675.00","$144,618.00",Los Angeles Unified School District
1,5554021048,West Hollywood,Austin,Allen,Ryan,Wiggins,1203,Sweetzer,1203 N Sweetzer Ave #205,205,...,0.0,"$4,647.02","$503,000.00",Conventional,Watermark Capital Inc,Fidelity National Title,"$360,220.00","$196,805.00","$163,415.00",Los Angeles Unified School District
2,5560022004,West Hollywood,NaN,1120 larrabee llc,NaN,NaN,1124,Larrabee,1124 Larrabee St,NaN,...,2.0,"$25,930.42",NaN,NaN,NaN,Fidelity Sherman Oaks,"$2,126,700.00","$1,769,700.00","$357,000.00",Los Angeles Unified School District
3,4339015122,West Hollywood,NaN,Opendoor Property Trust I,NaN,NaN,960,Larrabee,960 Larrabee St #207,207,...,0.0,"$7,067.67",NaN,NaN,NaN,North American Title Co Inc,"$572,219.00","$444,979.00","$127,240.00",Los Angeles Unified School District
4,4339009099,West Hollywood,Rozita,Payman,Justin,Slmantov,1010,Palm,1010 Palm Ave #305,305,...,0.0,"$6,287.98",NaN,NaN,NaN,None Available,"$499,500.00","$170,000.00","$329,500.00",Los Angeles Unified School District


In [3]:
# Create dataframe with addresses of all sales
address_df = df.loc[:, ["PARCEL", "SITUS STREET ADDRESS", "SITUS CITY", "SITUS STATE", 
                        "SITUS ZIP CODE"]].set_index("PARCEL")

In [4]:
address_df = address_df.dropna()

In [5]:
address_df["SITUS ZIP CODE"] = address_df["SITUS ZIP CODE"].astype(int)

In [6]:
address_df["FULL ADDRESS"] = address_df["SITUS STREET ADDRESS"] + ", " + \
                            address_df["SITUS CITY"] + ", " + \
                            address_df["SITUS STATE"] + " " + \
                            address_df["SITUS ZIP CODE"].astype(str)
address_df.head()

,SITUS STREET ADDRESS,SITUS CITY,SITUS STATE,SITUS ZIP CODE,FULL ADDRESS
PARCEL,,,,,
5559006020,8787 Shoreham Dr #101,West Hollywood,CA,90069,"8787 Shoreham Dr #101, West Hollywood, CA 90069"
5554021048,1203 N Sweetzer Ave #205,West Hollywood,CA,90069,"1203 N Sweetzer Ave #205, West Hollywood, CA 9..."
5560022004,1124 Larrabee St,West Hollywood,CA,90069,"1124 Larrabee St, West Hollywood, CA 90069"
4339015122,960 Larrabee St #207,West Hollywood,CA,90069,"960 Larrabee St #207, West Hollywood, CA 90069"
4339009099,1010 Palm Ave #305,West Hollywood,CA,90069,"1010 Palm Ave #305, West Hollywood, CA 90069"


In [7]:
# Define the geo locator
locator = Nominatim(user_agent="myGeocoder")

In [8]:
# Add a delay between geocoding calls to prevent service rejecting the geo call
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [10]:
# Create the location column applying the goe location
for index, row in address_df.iterrows():
    try:
        address_df["location"][index] = address_df["FULL ADDRESS"][index].apply(geocode)
    except:
        print(address_df["FULL ADDRESS"][index])
        print(index)

PARCEL
5559006020    8787 Shoreham Dr #101, West Hollywood, CA 90069
5559006020    8787 Shoreham Dr #101, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5559006020
1203 N Sweetzer Ave #205, West Hollywood, CA 90069
5554021048
PARCEL
5560022004    1124 Larrabee St, West Hollywood, CA 90069
5560022004    1124 Larrabee St, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560022004
960 Larrabee St #207, West Hollywood, CA 90069
4339015122
1010 Palm Ave #305, West Hollywood, CA 90069
4339009099
1037 N Laurel Ave #13, West Hollywood, CA 90046
5529024043
1250 N Harper Ave #415, West Hollywood, CA 90046
5554016103
906 N Doheny Dr #316, West Hollywood, CA 90069
4340024039
1400 N Sweetzer Ave #405, West Hollywood, CA 90069
5554019082
8535 W West Knoll Dr #208, West Hollywood, CA 90069
4339004057
1011 N Orange Grove Ave #1, West Hollywood, CA 90046
5530012028
1345 N Hayworth Ave #2, West Hollywood, CA 90046
5554004026
1155 N La Cienega Blvd #1001, West Hollywood, CA 90069
5

PARCEL
5560024042    1133 N Clark St #304, West Hollywood, CA 90069
5560024042    1133 N Clark St #304, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560024042
850 N Kings Rd #310, West Hollywood, CA 90069
5529004140
1426 N Laurel Ave #405, West Hollywood, CA 90046
5554003032
1133 N Formosa Ave #3, West Hollywood, CA 90046
5531007058
1037 N Laurel Ave #10, West Hollywood, CA 90046
5529024040
1330 N Crescent Heights Blvd #4, West Hollywood, CA 90046
5554006019
1155 N La Cienega Blvd #401, West Hollywood, CA 90069
5555006063
1152 N La Cienega Blvd #301, West Hollywood, CA 90069
5555004099
1112 N Olive Dr #6, West Hollywood, CA 90069
5554026142
851 N San Vicente Blvd #304, West Hollywood, CA 90069
4340006089
950 N Kings Rd #241, West Hollywood, CA 90069
5529025142
8121 Norton Ave #401, West Hollywood, CA 90046
5554009059
1250 N Kings Rd #407, West Hollywood, CA 90069
5554025099
1228 N La Cienega Blvd #104, West Hollywood, CA 90069
5555002134
8530 Holloway Dr #202, West Holly

PARCEL
5560023060    1129 Larrabee St #7, West Hollywood, CA 90069
5560023060    1129 Larrabee St #7, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560023060
PARCEL
5560023110    1201 Larrabee St #304, West Hollywood, CA 90069
5560023110    1201 Larrabee St #304, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560023110
8562 W West Knoll Dr #5, West Hollywood, CA 90069
4339005032
9020 Phyllis Ave, West Hollywood, CA 90069
4340023005
1275 Havenhurst Dr #10, West Hollywood, CA 90046
5554016042
8983 Norma Pl, West Hollywood, CA 90069
4340020005
8121 Norton Ave #403, West Hollywood, CA 90046
5554009061
950 N Kings Rd #232, West Hollywood, CA 90069
5529025133
1121 N Olive Dr #302, West Hollywood, CA 90069
5555003098
8909 Dorrington Ave, West Hollywood, CA 90048
4336013038
1200 N Flores St #214, West Hollywood, CA 90069
5554021108
8703 W West Knoll Dr #106, West Hollywood, CA 90069
4339008069
1016 Hancock Ave #6, West Hollywood, CA 90069
4339008052
7601 Lexington Av

PARCEL
5560023064    1129 Larrabee St #11, West Hollywood, CA 90069
5560023064    1129 Larrabee St #11, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560023064
8490 Fountain Ave #206, West Hollywood, CA 90069
5555004082
8400 De Longpre Ave #203, West Hollywood, CA 90069
5554023033
1330 N Crescent Heights Blvd #6, West Hollywood, CA 90046
5554006021
9025 Dicks St, West Hollywood, CA 90069
4340022041
8917 Rosewood Ave, West Hollywood, CA 90048
4336016018
8609 W West Knoll Dr D, West Hollywood, CA 90069
4339008058
999 N Doheny Dr #1208, West Hollywood, CA 90069
4340027159
1230 N Sweetzer Ave #309, West Hollywood, CA 90069
5554020085
1200 N Flores St #210, West Hollywood, CA 90069
5554021104
1233 N Laurel Ave #110, West Hollywood, CA 90046
5554010042
999 N Doheny Dr #105, West Hollywood, CA 90069
4340027029
944 Palm Ave #2, West Hollywood, CA 90069
4339009164
1100 Alta Loma Rd #4, West Hollywood, CA 90069
5555005046
818 N Doheny Dr #1402, West Hollywood, CA 90069
4340018130
8

PARCEL
5560024013    1222 Hilldale Ave, West Hollywood, CA 90069
5560024013    1222 Hilldale Ave, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560024013
964 Larrabee St #207, West Hollywood, CA 90069
4339015061
8960 Cynthia St #302, West Hollywood, CA 90069
4340021065
723 Westmount Dr #307, West Hollywood, CA 90069
4337008155
PARCEL
5560022042    1145 Horn Ave D, West Hollywood, CA 90069
5560022042    1145 Horn Ave D, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5560022042
1260 N Kings Rd #3, West Hollywood, CA 90069
5554025142
1211 N Vista St, West Hollywood, CA 90046
5531001023
845 N Crescent Heights Blvd, Los Angeles, CA 90046
5529016046
906 N Doheny Dr #318, West Hollywood, CA 90069
4340024040
960 Larrabee St #222, West Hollywood, CA 90069
4339015137
906 N Doheny Dr #214, West Hollywood, CA 90069
4340024034
1131 Alta Loma Rd #113, West Hollywood, CA 90069
5559001034
1003 Hancock Ave, West Hollywood, CA 90069
4339009008
1860 N La Cienega Blvd #200, West 

...

PARCEL
5559006108    8787 Shoreham Dr #1001, West Hollywood, CA 90069
5559006108    8787 Shoreham Dr #1001, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5559006108
2501 Outpost Dr, Los Angeles, CA 90068
5572007001
8459 Ridpath Dr, Los Angeles, CA 90046
5567025037
8036 Woodrow Wilson Dr, Los Angeles, CA 90046
5570030035
3324 Oakley Dr, Los Angeles, CA 90068
2425021014
1368 Doheny Pl, Los Angeles, CA 90069
5560010023
9318 Warbler Way, Los Angeles, CA 90069
5561014013
2181 Outpost Dr, Los Angeles, CA 90068
5572014012
1735 N Fuller Ave #303, Los Angeles, CA 90046
5550032072
6760 Hillpark Dr #507, Los Angeles, CA 90068
5549004194
3144 Barbara Ct, Los Angeles, CA 90068
2429013002
1200 N Wetherly Dr, Los Angeles, CA 90069
5560029025
2580 Greenvalley Rd, Los Angeles, CA 90046
5564032031
3149 Ellington Dr, Los Angeles, CA 90068
2429003008
1547 Sunset Plaza Dr, Los Angeles, CA 90069
5558007007
8558 Walnut Dr, Los Angeles, CA 90046
5556013014
8501 Hedges Pl, West Hollywood, CA 90069

PARCEL
5559006088    8787 Shoreham Dr #801, West Hollywood, CA 90069
5559006088    8787 Shoreham Dr #801, West Hollywood, CA 90069
Name: FULL ADDRESS, dtype: object
5559006088
7180 Woodrow Wilson Dr, Los Angeles, CA 90068
2428001030
8556 Franklin Ave, Los Angeles, CA 90069
5558022003
1133 Cory Ave, West Hollywood, CA 90069
5560035017
1940 N Highland Ave #17, Los Angeles, CA 90068
5575024033
6708 Hillpark Dr #104, Los Angeles, CA 90068
5549004052
9208 Cordell Dr, Los Angeles, CA 90069
4392010021
1680 N Doheny Dr, Los Angeles, CA 90069
5561006041
7290 Packwood Trl, Los Angeles, CA 90068
2428015016
8561 Hillside Ave, Los Angeles, CA 90069
5558030013
7250 Franklin Ave #1106, Los Angeles, CA 90046
5548025191
1586 Sunset Plaza Dr, Los Angeles, CA 90069
5558011013
9342 Sierra Mar Dr, Los Angeles, CA 90069
4392017013
8623 Franklin Ave, Los Angeles, CA 90069
5558021018
1705 Queens Ct, Los Angeles, CA 90069
5558021016
8831 Lookout Mountain Ave, Los Angeles, CA 90046
5563028004
2641 Nichols Canyo

6480 Odin St, Los Angeles, CA 90068
5576004020
2217 Canyon Dr, Los Angeles, CA 90068
5586021003
2811 Belden Dr, Los Angeles, CA 90068
5582014011
3124 Belden Dr, Los Angeles, CA 90068
5583009006
5860 Spring Oak Dr, Los Angeles, CA 90068
5580017029
6233 Mulholland Hwy, Los Angeles, CA 90068
5581018010
2140 N Bronson Ave, Los Angeles, CA 90068
5586022001
2553 Glen Green St, Los Angeles, CA 90068
5580003015
2338 El Contento Dr, Los Angeles, CA 90068
5585029028
3480 Barham Blvd #116, Los Angeles, CA 90068
5577007105
2516 Hollyridge Dr, Los Angeles, CA 90068
5580010004
2915 Lakeridge Dr, Los Angeles, CA 90068
5577022004
3086 Durand Dr, Los Angeles, CA 90068
5582011007
2748 Hollyridge Dr, Los Angeles, CA 90068
5580016003
2222 N Beachwood Dr #215, Los Angeles, CA 90068
5586011053
6232 Glen Airy St, Los Angeles, CA 90068
5585024013
2461 Creston Way, Los Angeles, CA 90068
5585017006
2700 Cahuenga Blvd E #2302, Los Angeles, CA 90068
5577010058
6358 La Rocha Dr, Los Angeles, CA 90068
5576008034
34

6380 Drexel Ave, Los Angeles, CA 90048
5510009018
6440 Drexel Ave, Los Angeles, CA 90048
5510008028
6510 Drexel Ave, Los Angeles, CA 90048
5510008023
6646 Drexel Ave, Los Angeles, CA 90048
5510007027
6670 Drexel Ave, Los Angeles, CA 90048
5510007022
6671 Drexel Ave, Los Angeles, CA 90048
5510004015
6721 Drexel Ave, Los Angeles, CA 90048
5510005005
618 S Dunsmuir Ave, Los Angeles, CA 90036
5508010016
625 S Dunsmuir Ave, Los Angeles, CA 90036
5508011009
935 S Dunsmuir Ave, Los Angeles, CA 90036
5089025025
939 S Dunsmuir Ave, Los Angeles, CA 90036
5089025026
1021 S Dunsmuir Ave, Los Angeles, CA 90019
5085009004
1052 S Dunsmuir Ave, Los Angeles, CA 90019
5085011002
1057 S Dunsmuir Ave, Los Angeles, CA 90019
5085009040
5418 Edgewood Pl, Los Angeles, CA 90019
5085012019
112 N Edinburgh Ave, Los Angeles, CA 90048
5511002019
129 N Edinburgh Ave, Los Angeles, CA 90048
5511003006
327 N Edinburgh Ave, Los Angeles, CA 90048
5527034020
428 N Edinburgh Ave, Los Angeles, CA 90048
5527032008
429 N Edi

1154 S Point View St, Los Angeles, CA 90035
5087023022
1157 S Point View St, Los Angeles, CA 90035
5087020012
1242 S Point View St, Los Angeles, CA 90035
5087022029
1107 S Redondo Blvd, Los Angeles, CA 90019
5084019016
1280 S Redondo Blvd, Los Angeles, CA 90019
5070005034
1342 S Redondo Blvd, Los Angeles, CA 90019
5070006023
600 S Ridgeley Dr, Los Angeles, CA 90036
5508012044
600 S Ridgeley Dr, Los Angeles, CA 90036
5508012042
600 S Ridgeley Dr, Los Angeles, CA 90036
5508012058
600 S Ridgeley Dr, Los Angeles, CA 90036
5508012069
600 S Ridgeley Dr, Los Angeles, CA 90036
5508012083
644 S Ridgeley Dr, Los Angeles, CA 90036
5508012020
648 S Ridgeley Dr, Los Angeles, CA 90036
5508012031
914 S Ridgeley Dr, Los Angeles, CA 90036
5089026017
1014 S Ridgeley Dr, Los Angeles, CA 90019
5085008036
1021 S Ridgeley Dr, Los Angeles, CA 90019
5085007022
1031 S Ridgeley Dr, Los Angeles, CA 90019
5085007024
1126 S Ridgeley Dr, Los Angeles, CA 90019
5085008011
1212 S Ridgeley Dr, Los Angeles, CA 90019
508

In [ ]:
# Create longitude, laatitude and altitude from location column (returns tuple)
address_df['point'] = sample_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
# Split point column into latitude, longitude and altitude columns
address_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(sample_df['point'].tolist(), index=sample_df.index)

In [ ]:
final_address_df = address_df.drop(columns=["FULL ADDRESS","location","point","altitude"],axis=1)
final_address_df.head()

In [ ]:
final_address_df.to_csv('address_loc.csv')

In [ ]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [ ]:
cols = ['Site Address']
geojson = df_to_geojson(address_df, cols)

In [ ]:
geojson

In [ ]:
json = json.dumps(geojson)
f = open("output.json","w")
f.write(json)
f.close()